In [ ]:
import numpy as np
import random
from collections import deque

import gym
import torch
import seaborn as sns
from tqdm.notebook import tqdm

from torch import nn
import pandas as pd
from torch import optim
from typing import Any
from copy import deepcopy
# from gym.wrappers import Monitor

import gym
from gym import spaces
from gym.utils import seeding

from collections import namedtuple
from dataclasses import dataclass

from matplotlib import pyplot as plt


In [ ]:
%matplotlib inline
import numpy as np
import random
import time
import os
import gym
import json
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd


from copy import deepcopy
from tqdm.notebook import tqdm
from dataclasses import dataclass
from matplotlib import animation
from IPython.display import HTML
from typing import Any
from collections import deque

mpl.rcParams['figure.dpi']= 100

# Plotting

In [ ]:
def plot(logs, x_key, y_key, legend_key, **kwargs):
    nums = len(logs[legend_key].unique())
    palette = sns.color_palette("hls", nums)
    if 'palette' not in kwargs:
        kwargs['palette'] = palette
    sns.lineplot(x=x_key, y=y_key, data=logs, hue=legend_key, **kwargs)

def set_random_seed(seed):
    np.random.seed(seed)
    random.seed(seed)

# set random seed
seed = 0
set_random_seed(seed=seed)

# NChain Env

In [ ]:
class NChainEnv(gym.Env):
    """n-Chain environment

    This game presents moves along a linear chain of states, with two actions:
     0) forward, which moves along the chain but returns no reward
     1) backward, which returns to the beginning and has a small reward

    The end of the chain, however, presents a large reward, and by moving
    'forward' at the end of the chain this large reward can be repeated.

    At each action, there is a small probability that the agent 'slips' and the
    opposite transition is instead taken.

    The observed state is the current state in the chain (0 to n-1).

    This environment is described in section 6.1 of:
    A Bayesian Framework for Reinforcement Learning by Malcolm Strens (2000)
    http://ceit.aut.ac.ir/~shiry/lecture/machine-learning/papers/BRL-2000.pdf
    """
    def __init__(self, n=20, slip=0.0, small=0, large=1, ):
        self.n = n
        self.slip = slip  # probability of 'slipping' an action
        self.small = small  # payout for 'backwards' action
        self.large = large  # payout at end of chain for 'forwards' action
        self.state = 0  # Start at beginning of the chain
        self.action_space = spaces.Discrete(2)
        self.observation_space = spaces.Discrete(self.n)
        self.time_step = 0
        self.max_time_step = 100
        self.info = False

    def step(self, action):
        self.time_step += 1
        assert self.action_space.contains(action)
        reward = 0
        if np.random.rand() < self.slip:
            action = not action  # agent slipped, reverse action taken
        if action:  # 'backwards': go back to the beginning, get small reward
#             reward = self.small
            if(self.state > 0):
                self.state = self.state - 1
            
        else:
            assert (self.state < (self.n-1))
            self.state = self.state +1
#             reward = 1
            if(self.state == (self.n-1)):
                reward = self.large 
        
        done = (self.state == (self.n-1)) or (self.time_step == self.max_time_step)
#         reward = self.large if (self.state == (self.n-1)) else 0
        if (self.time_step == self.max_time_step):
            self.info = True
        return self.state, reward, done, self.info

    def reset(self):
        self.time_step = 0
        self.state = 0
        self.info = False
        return self.state
    
#     def action_sample(self):
#         return self.action_space.sample()
    

# Buffers : Cyclic and Graph

In [ ]:
from buffer import ReplayBufferGraph, PrioritizedReplayBuffer, CyclicBuffer

# Q Learning Agent

In [ ]:
@dataclass
class QLearningAgent:
    env: gym.Env #
    learning_rate: float #
    gamma: float#
    initial_epsilon: float#
    min_epsilon: float#
    max_decay_episodes: int#
    capacity: int#
    batch_size: int
    warmup_steps: int
    init_q_value: float
    verbose_buffer: bool
    buffer_type: str
    eta: float

    def __post_init__(self):
        self.num_states = self.env.observation_space.n
        self.num_actions = self.env.action_space.n
        self.reset()
        self.ground_truth = np.array([[0.83451376, 0.82616862],
                                   [0.84294319, 0.82616862],
                                   [0.85145777, 0.83451376],
                                   [0.86005835, 0.84294319],
                                   [0.86874581, 0.85145777],
                                   [0.87752102, 0.86005835],
                                   [0.88638487, 0.86874581],
                                   [0.89533825, 0.87752102],
                                   [0.90438208, 0.88638487],
                                   [0.91351725, 0.89533825],
                                   [0.92274469, 0.90438208],
                                   [0.93206535, 0.91351725],
                                   [0.94148015, 0.92274469],
                                   [0.95099005, 0.93206535],
                                   [0.96059601, 0.94148015],
                                   [0.970299  , 0.95099005],
                                   [0.9801    , 0.96059601],
                                   [0.99      , 0.970299  ],
                                   [1.        , 0.9801    ],
                                   [0.        , 0.        ]])

    def decay_epsilon(self):
        ### TODO: decay epsilon by ep_reduction while respecting min_epsilon ################
        ###.      this function is called every episode
        self.epsilon = max(self.min_epsilon, self.epsilon - self.ep_reduction)

        #####################################################################
    
    def reset(self):
#         print("eta", self.eta)
        if self.buffer_type == 'cyclic':
            self.buffer = CyclicBuffer(self.capacity)
        else:
            self.buffer = ReplayBufferGraph(max_transitions = self.capacity, 
                                            verbose = self.verbose_buffer,
                                            vertex_dim = 1, 
                                            state_dim = 1, 
                                            projection_matrix = np.array([[1]]))
            
            self.per_buffer = PrioritizedReplayBuffer(self.capacity)
            
            
    
        self.epsilon = self.initial_epsilon
        self.ep_reduction = (self.epsilon - self.min_epsilon) / float(self.max_decay_episodes)
        self.Q = np.random.rand(self.num_states, self.num_actions)*self.init_q_value #np.ones((self.num_states, self.num_actions)) 

    def update_Q(self):#, state, action, reward, next_state, done):
        ### TODO: update self.Q given new experience. #######################

        if self.buffer_type == 'cyclic':
            if len(self.buffer) < self.warmup_steps:
                return False, np.sum(abs(self.ground_truth-self.Q))/40
        
        else:
            if len(self.buffer.batch_queue) < self.batch_size:
                return False, np.sum(abs(self.ground_truth-self.Q))/40
            if (self.eta > 1e-5) and (len(self.per_buffer) < self.batch_size):
                return False, np.sum(abs(self.ground_truth-self.Q))/40
        
        if self.buffer_type == 'cyclic' or (np.random.rand() >= self.eta):
            state, action, reward, next_state, done = self.buffer.sample(self.batch_size)
            sampled_per = False
        else:
            sampled_per = True
            state, action, reward, next_state, done, idx, is_weight = self.per_buffer.sample(self.batch_size)

        
        state = state[0]
        action = action[0]
        reward = reward[0]
        next_state = next_state[0]
        done = done[0]
        # compute target
        y = reward + self.gamma*np.amax(self.Q[next_state,:])
        # compute error
        e = y - self.Q[state, action]
        # update Q
        
        self.Q[state, action] = self.Q[state, action] + self.learning_rate * e
        
        if done:
            self.Q[next_state, :] = 0
            self.Q[state, action] = reward
            
        if sampled_per:
            self.per_buffer.update_priorities(idx, [max(abs(e), 0.001)])
            
#       print("value error", np.sum(abs(self.ground_truth-self.Q))/40)
        return True, np.sum(abs(self.ground_truth-self.Q))/40

        #####################################################################
    def append_sample_to_per(self, state, action, reward, next_state, done):
        self.per_buffer.add(state, action, reward, next_state, done)
    
    def get_action(self, state = None, choose_random = False, greedy_only = False):
        
        if (greedy_only):
            m = max(self.Q[state, :])
            index = [i for i, j in enumerate(self.Q[state, :]) if j == m]
#             print("index", index)
            best_action = np.random.choice(index)
            chosen_action = best_action
            return chosen_action


        ### TODO: select an action given self.Q and self.epsilon ############
        random_num = np.random.uniform(0., 1.)
        if choose_random or random_num < self.epsilon:
            chosen_action = np.random.choice(np.arange(self.num_actions))
#             print(f'randomly chosen action {chosen_action}')
        else:
            m = max(self.Q[state, :])
            index = [i for i, j in enumerate(self.Q[state, :]) if j == m]
#             print("index", index)
            best_action = np.random.choice(index)
            chosen_action = best_action
#             print(f'best action {chosen_action}')
        return chosen_action

# Q learning Engine

In [ ]:
@dataclass
class QLearningEngine:
    env: gym.Env
    agent: Any
    smooth_len: int
    max_episodes: int
    max_updates_record: int
        
    def test(self, env=None, render=False):
        rewards = []
        actions_outer = []
        for i in range(10):
            actions = []
            env = self.env if env is None else env
            ob = env.reset()
            ret = 0
            while True:
                if render:
                    env.render()
                action = self.agent.get_action(ob, greedy_only=True)
                actions.append(action)
                next_ob, reward, done, info = env.step(action)
                ret += reward
                ob = next_ob
                if done:
                    rewards.append(ret)
                    actions_outer.append(actions)
                    break
#         print(actions_outer)
        return np.mean(rewards)
    
    def run(self, n_runs=1):
        rewards = []
        log = []
        log_updates = []
                
        for i in tqdm(range(n_runs), desc='Runs'):
            total_updates = 0
            self.agent.reset()
            state = self.env.reset()
            
            smooth_ep_return = deque(maxlen=self.smooth_len)
            ep_rewards = []
            
            ep_current_rewards = []
            ep_steps = []
            ep_updates = []
            
            smooth_test_returns = deque(maxlen=self.smooth_len)
            test_returns = []
            
            smooth_value_errors = deque(maxlen=self.smooth_len)
            value_errors = []
            per_update_value_error = []

            ret = 0
            num_ep = 0
                       
            for t in tqdm(range(self.max_episodes), desc='Episode'):
                
                if t < self.agent.warmup_steps:
                    action = self.env.action_space.sample()
                else:
                    action = self.agent.get_action(state)
                
                next_state, reward, done, info = self.env.step(action)
                true_done = done and not info
                
  
                if self.agent.buffer_type == 'cyclic':
                    self.agent.buffer.append((state, action, reward, next_state, true_done))
                    self.agent.update_Q()
                    total_updates += 1
                else:
                    if true_done:
                        self.agent.buffer.add_to_terminal_vertices(next_state)
                        
                    state_array = np.array([[state]])
                    next_state_array = np.array([[next_state]])
                    self.agent.buffer.append((state_array, action, reward, next_state_array, true_done, t))
                    self.agent.per_buffer.append((state_array, action, reward, next_state_array, true_done))
#                     self.agent.per_buffer.append((state, action, reward, next_state, true_done))

                    ########################################################            
                    if (self.agent.eta >= 1) or len(self.agent.buffer.terminal_vertices) > 0:
                        if self.agent.eta < 1:
                            while (t > self.agent.warmup_steps) and (len(self.agent.buffer.batch_queue) < self.agent.batch_size):
                                result = self.agent.buffer.step_reverse_BFS()
        #                         print('\n')
                                if not result:
                                    break

                        if t > self.agent.warmup_steps:
                            success, value_error = self.agent.update_Q()
                            total_updates += int(success)
     
                    ########################################################
                
                ret += reward
                state = next_state
                
                if done:
                    
                    test_ret = self.test()
#                         print(f'Step:{t} Testing Return: {test_ret}, Total updates:{total_updates}')
                        
                    state = self.env.reset()
                    ep_current_rewards.append(ret)
                    smooth_ep_return.append(ret)
                    ep_rewards.append(np.mean(smooth_ep_return))
                    ep_steps.append(t)
                    ep_updates.append(total_updates)
                    smooth_test_returns.append(test_ret)
                    test_returns.append(np.mean(smooth_test_returns))
                    value_errors.append(np.mean(abs(self.agent.ground_truth-self.agent.Q)))
#                     smooth_value_errors.append(value_error)
#                     value_errors.append(np.mean(smooth_value_errors))
                        
                    ret = 0
                    num_ep += 1

                self.agent.decay_epsilon()
                
            interp_test_returns = np.interp(np.arange(self.max_episodes), ep_updates, test_returns)
            interp_value_errors = np.interp(np.arange(self.max_episodes), ep_updates, value_errors)
            run_log2 = pd.DataFrame({'Test return': interp_test_returns[:self.max_updates_record],
                                     'Value error': interp_value_errors[:self.max_updates_record],
                                     'Number of updates': np.arange(self.max_episodes)[:self.max_updates_record],
                                     'buffer_type': self.agent.buffer_type,
                                     'iqv': self.agent.init_q_value,
                                     'eta':str(self.agent.eta)})
            
            run_log = pd.DataFrame({'return': ep_rewards, 
                                    'current_rewards': ep_current_rewards,
                                    'steps': ep_steps,
                                    'num_updates': ep_updates,
                                    'episode': np.arange(len(ep_rewards)), 
                                    'test_rewards': test_returns,
                                    'value_errors': value_errors,
                                    'buffer_type': self.agent.buffer_type,
                                    'iqv': self.agent.init_q_value,
                                    'eta':str(self.agent.eta)})
            log.append(run_log)
            log_updates.append(run_log2)
            
            
        return log, log_updates

    
    
def qlearning_sweep_eta(init_q_value, buffers = ['ter'], n_runs=4, smooth_len = 10, 
                    max_episodes=100000, epsilon=0.9, learning_rate=0.99, max_decay_epsiodes = 1000,
                    max_updates_record = 100,
                    T_warmup = 100, batch_size = 1, capacity = 10000, etas = [0.1]):
    logs = dict()
    agents = []
    seed = 0
    env = NChainEnv()    

    configs = {'env':env,
               'learning_rate':learning_rate,
               'gamma':0.99,
               'warmup_steps':T_warmup,
               'capacity':capacity,
               'initial_epsilon':epsilon,
               'min_epsilon':0.01,
               'batch_size' : batch_size,
               'max_decay_episodes':max_decay_epsiodes,
               'init_q_value':init_q_value,
               'verbose_buffer':False}
    
    for buffer in buffers:
        if buffer == 'ter':
            for eta in etas:
                set_random_seed(seed=seed)
                agent = QLearningAgent(**configs, eta = eta, buffer_type = buffer)
                engine = QLearningEngine(env=env, agent=agent, 
                                         max_episodes=max_episodes, 
                                         smooth_len = smooth_len,
                                        max_updates_record = max_updates_record)
                _, ep_log = engine.run(n_runs)
                ep_log = pd.concat(ep_log, ignore_index=True)
                logs[str(eta)] = ep_log
                agents.append(agent)
        else:
            eta = buffer
            set_random_seed(seed=seed)
            agent = QLearningAgent(**configs, eta = eta, buffer_type = buffer)
            engine = QLearningEngine(env=env, agent=agent, 
                                     max_episodes=max_episodes, 
                                     smooth_len = smooth_len,
                                    max_updates_record = max_updates_record)
            _, ep_log = engine.run(n_runs)
            ep_log = pd.concat(ep_log, ignore_index=True)
            logs[str(eta)] = ep_log
            agents.append(agent)
            
    logs = pd.concat(logs, ignore_index=True)
    
    return logs, agents




# Main Function

In [ ]:

buffers = ['ter', 'cyclic']
etas = [0.001, 0.1, 0.99]

num_plots = 4

eps_logs, eps_agents = qlearning_sweep_eta(init_q_value=0.1, 
                                           n_runs=3,
                                           max_updates_record=200,
                                           buffers = buffers,
                                           max_episodes=5000,
                                           max_decay_epsiodes=5000,
                                           epsilon=0.99, 
                                           T_warmup = 40, 
                                           batch_size = 1, 
                                           smooth_len = 1,
                                           capacity = 1000, etas = etas)



In [ ]:
plot(eps_logs, x_key='Number of updates', y_key='Test return', legend_key='eta', estimator='mean', ci='sd', palette = sns.color_palette("hls", 4))


In [ ]:
plot(eps_logs, x_key='Number of updates', y_key='Value error', legend_key='eta', estimator='mean', ci='sd', palette = sns.color_palette("hls", 4))


In [ ]:
pip install seaborn --upgrade

In [ ]:
plot(eps_logs, x_key='episode', y_key='test_rewards', legend_key='eta', estimator='mean', ci='sd', palette = sns.color_palette("hls", 3))


In [ ]:
plot(eps_logs, x_key='num_updates', y_key='value_errors', legend_key='eta', estimator='mean', ci='sd', palette = sns.color_palette("hls", 3))
